# QRコード検出

Opencvのインストールを行うコマンドが入っています。一度実行した後はこの処理はスキップ可能です。初回インストール後にkernelの再起動が必要です。

In [ ]:
!pip install -q opencv-python-headless

In [ ]:
%run setup_demo_env.py

In [ ]:
import cv2
import grpc
import kachaka_api_pb2
import numpy as np
from IPython.display import Image, clear_output, display
from kachaka_api_pb2_grpc import KachakaApiStub

In [ ]:
stub = KachakaApiStub(grpc.aio.insecure_channel(kachaka_api_server))

以下実行後、スマートフォンなどで https://pf-robotics.github.io/textcode/ にアクセスし、好きな英数字を入力して表示されたQRコードをカチャカの前カメラの前に近付けて下さい。

In [ ]:
req = kachaka_api_pb2.GetRequest()
qcd = cv2.QRCodeDetector()

while True:
    resp = await stub.GetFrontCameraRosCompressedImage(req)
    image = cv2.imdecode(np.frombuffer(resp.image.data, dtype=np.uint8), flags=1)
    decoded_info, corners, _ = qcd.detectAndDecode(image)
    if corners is not None:
        image = cv2.polylines(image, corners.astype(int), True, (0, 0, 255), 2)
    if decoded_info != "":
        image = cv2.putText(
            image,
            decoded_info,
            corners[0][0].astype(int),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (0, 0, 255),
            2,
            cv2.LINE_AA,
        )
    _, ret = cv2.imencode(
        ".jpg", cv2.resize(image, (int(image.shape[1] / 2), int(image.shape[0] / 2)))
    )
    clear_output(wait=True)
    display(Image(data=ret, format="jpeg"))
    req.metadata.cursor = resp.metadata.cursor